In [7]:
needle_model_name = "NeedleModel"
sequence_browser_name = "NeedleCapture_01_SequenceBrowser"
image_name = "Image_Image"
image_copy1_name = "Image_Image_1"
image_copy2_name = "Image_Image_2"
segmentation_name = "Segmentation"
segmentation_copy1_name = "Segmentation_1"
segmentation_copy2_name = "Segmentation_2"

output_folder = "C:\Data\Temp"
filename_prefix = "NeedleCapture_01"

In [8]:
import datetime
import numpy as np
import os

In [14]:
browser_node = slicer.util.getFirstNodeByName(sequence_browser_name, className='vtkMRMLSequenceBrowserNode')
model_node = slicer.util.getFirstNodeByName(needle_model_name, className="vtkMRMLModelNode")
image_node = slicer.util.getFirstNodeByName(image_name, className="vtkMRMLScalarVolumeNode")
image_copy1_node = slicer.util.getFirstNodeByName(image_copy1_name, className="vtkMRMLScalarVolumeNode")
image_copy2_node = slicer.util.getFirstNodeByName(image_copy2_name, className="vtkMRMLScalarVolumeNode")
segmentation_node = slicer.util.getFirstNodeByName(segmentation_name, className="vtkMRMLSegmentationNode")
segmentation_copy1_node = slicer.util.getFirstNodeByName(segmentation_copy1_name, className="vtkMRMLSegmentationNode")
segmentation_copy2_node = slicer.util.getFirstNodeByName(segmentation_copy2_name, className="vtkMRMLSegmentationNode")


if browser_node is None:
    logging.error("Could not find input browser node: {}".format(sequence_browser_name))
    raise

if model_node is None:
    logging.error("Could not find input model node: {}".format(needle_model_name))
    raise

if image_node is None:
    logging.error("Could not find input image node: {}".format(image_name))
    raise
    
if image_copy1_node is None:
    logging.error("Could not find input image node: {}".format(image_copy1_name))
    raise
    
if image_copy2_node is None:
    logging.error("Could not find input image node: {}".format(image_copy2_name))
    raise

if segmentation_node is None:
    logging.error("Could not find input segmentation node: {}".format(segmentation_name))
    raise

if segmentation_copy1_node is None:
    logging.error("Could not find input segmentation node: {}".format(segmentation_copy1_name))
    raise
    
if segmentation_copy2_node is None:
    logging.error("Could not find input segmentation node: {}".format(segmentation_copy2_name))
    raise

In [15]:
labelmap_name = "temp_labelmap"
labelmap_node = slicer.util.getFirstNodeByName(labelmap_name, className="vtkMRMLLabelMapVolumeNode")
if labelmap_node is None:
    labelmap_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    labelmap_node.SetName(labelmap_name)
    
labelmap_copy1_name = "temp_labelmap_copy1"
labelmap_copy1_node = slicer.util.getFirstNodeByName(labelmap_copy1_name, className="vtkMRMLLabelMapVolumeNode")
if labelmap_copy1_node is None:
    labelmap_copy1_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    labelmap_copy1_node.SetName(labelmap_copy1_name)
    
labelmap_copy2_name = "temp_labelmap_copy2"
labelmap_copy2_node = slicer.util.getFirstNodeByName(labelmap_copy2_name, className="vtkMRMLLabelMapVolumeNode")
if labelmap_copy2_node is None:
    labelmap_copy2_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
    labelmap_copy2_node.SetName(labelmap_copy2_name)

In [16]:
import vtkSegmentationCorePython as vtkSegmentationCore

def removeSegments(selectedSegmentation):
    num_segments = selectedSegmentation.GetSegmentation().GetNumberOfSegments()
    for i in range(num_segments):
        segmentId = selectedSegmentation.GetSegmentation().GetNthSegmentID(i)
        selectedSegmentation.RemoveSegment(segmentId)

In [17]:
browser_node.SelectFirstItem()

segmentation_logic = slicer.modules.segmentations.logic()

png_writer = vtk.vtkPNGWriter()

image_cast = vtk.vtkImageCast()
image_cast.SetOutputScalarTypeToUnsignedChar()
image_cast.Update()

image_cast_copy1 = vtk.vtkImageCast()
image_cast_copy1.SetOutputScalarTypeToUnsignedChar()
image_cast_copy1.Update()

removeSegments(segmentation_node)
n = browser_node.GetNumberOfItems()
for i in range(n):
    
    #Get segmented image in US position
    segmentation_logic.ImportModelToSegmentationNode(model_node, segmentation_node)
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentation_node, labelmap_node, image_node)
    segmented_image = labelmap_node.GetImageData()
    
    #Get segmented image in front of US position
    segmentation_logic.ImportModelToSegmentationNode(model_node, segmentation_copy1_node)
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentation_copy1_node, labelmap_copy1_node, image_copy1_node)
    segmented_image_copy1 = labelmap_copy1_node.GetImageData()
    #Get segmented image behind US position
    segmentation_logic.ImportModelToSegmentationNode(model_node, segmentation_copy2_node)
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentation_copy2_node, labelmap_copy2_node, image_copy2_node)
    segmented_image_copy2 = labelmap_copy2_node.GetImageData()
    
    
    #Set ultrasound image filename
    img_file_name = filename_prefix + "_%04d_ultrasound" % i + ".png"
    ultrasound_fullname = os.path.join(output_folder, img_file_name)
    
    #Set segmentation image filenames
    seg_file_name = filename_prefix + "_%04d_original_segmentation" % i + ".png"
    segmentation_fullname = os.path.join(output_folder, seg_file_name)
    seg_copy1_file_name = filename_prefix + "_%04d_forward_segmentation" % i + ".png"
    segmentation_copy1_fullname = os.path.join(output_folder, seg_copy1_file_name)
    seg_copy2_file_name = filename_prefix + "_%04d_backward_segmentation" % i + ".png"
    segmentation_copy2_fullname = os.path.join(output_folder, seg_copy2_file_name)
    
    #Output ultrasound image to file
    png_writer.SetInputData(image_node.GetImageData())
    png_writer.SetFileName(ultrasound_fullname)
    png_writer.Update()
    png_writer.Write()
    
    #Output segmentations
    image_cast.SetInputData(segmented_image)
    image_cast.Update()
    png_writer.SetInputData(image_cast.GetOutput())
    png_writer.SetFileName(segmentation_fullname)
    png_writer.Update()
    png_writer.Write()

    
    image_cast_copy1.SetInputData(segmented_image_copy1)
    image_cast_copy1.Update()
    png_writer.SetInputData(image_cast_copy1.GetOutput())
    png_writer.SetFileName(segmentation_copy1_fullname)
    png_writer.Update()
    png_writer.Write()

    image_cast.SetInputData(segmented_image_copy2)
    image_cast.Update()
    png_writer.SetInputData(image_cast.GetOutput())
    png_writer.SetFileName(segmentation_copy2_fullname)
    png_writer.Update()
    png_writer.Write()
    

    removeSegments(segmentation_node)
    removeSegments(segmentation_copy1_node)
    removeSegments(segmentation_copy2_node)
    
    slicer.app.processEvents()
    browser_node.SelectNextItem()